In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
# df = pd.read_csv(file_path + 'Oanda_Aud_Usd_M5_2021-2022.csv')
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2021-2022.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

# df_long = pd.read_csv(file_path + 'Oanda_Aud_Usd_H1_2021-2022.csv')
df_long = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M30_2021-2022.csv')
df_long.Date = pd.to_datetime(df_long.Date)
df_long.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-01-03 22:00:00   1.36452   1.36495  1.36442    1.36461   1.36602   
1 2021-01-03 22:05:00   1.36470   1.36487  1.36448    1.36470   1.36620   
2 2021-01-03 22:10:00   1.36436   1.36491  1.36436    1.36488   1.36575   
3 2021-01-03 22:15:00   1.36498   1.36504  1.36496    1.36496   1.36549   
4 2021-01-03 22:20:00   1.36502   1.36613  1.36502    1.36544   1.36572   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   1.36638  1.36557    1.36611   1.36527   1.36563  1.36500    1.36536   
1   1.36628  1.36555    1.36555   1.36545   1.36553  1.36506    1.36512   
2   1.36576  1.36544    1.36568   1.36506   1.36534  1.36506    1.36528   
3   1.36559  1.36549    1.36556   1.36524   1.36532  1.36524    1.36526   
4   1.36691  1.36572    1.36641   1.36537   1.36652  1.36537    1.36592   

   Volume  
0      23  
1      11  
2       9  
3       4  
4      42  
--------------------------

In [5]:
print(df_long.head())
print('-' * 75)
print(df_long.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-01-03 22:00:00   1.36452   1.36619  1.36436    1.36522   1.36602   
1 2021-01-03 22:30:00   1.36536   1.36606  1.36525    1.36582   1.36629   
2 2021-01-03 23:00:00   1.36560   1.36687  1.36528    1.36682   1.36710   
3 2021-01-03 23:30:00   1.36680   1.36829  1.36669    1.36798   1.36702   
4 2021-01-04 00:00:00   1.36800   1.36827  1.36649    1.36655   1.36821   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   1.36691  1.36544    1.36658   1.36527   1.36652  1.36500    1.36590   
1   1.36681  1.36612    1.36670   1.36582   1.36640  1.36568    1.36626   
2   1.36718  1.36590    1.36704   1.36635   1.36699  1.36576    1.36693   
3   1.36850  1.36693    1.36818   1.36691   1.36838  1.36682    1.36808   
4   1.36845  1.36671    1.36675   1.36810   1.36836  1.36662    1.36665   

   Volume  
0     108  
1      38  
2     370  
3     461  
4    1045  
--------------------------

In [6]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

In [7]:
# Add technical indicators (for additional features)
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['ema100'] = pd.Series.ewm(df['Mid_Close'], span=100).mean()
df['ema50'] = pd.Series.ewm(df['Mid_Close'], span=50).mean()
df['ema25'] = pd.Series.ewm(df['Mid_Close'], span=25).mean()

df['atr'] = atr(df)
df['rsi'] = rsi(df)
df['rsi_sma'] = df['rsi'].rolling(50).mean()
df['adx'] = adx(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['macd'] = pd.Series.ewm(df['Mid_Close'], span=12).mean() - pd.Series.ewm(df['Mid_Close'], span=26).mean()
df['macdsignal'] = pd.Series.ewm(df['macd'], span=9).mean()
df['n_macd'], df['n_macdsignal'] = n_macd(df['macd'], df['macdsignal'])
df['slowk'], df['slowd'] = stoch(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['slowk_rsi'], df['slowd_rsi'] = stoch_rsi(df['rsi'])

df['sar'] = psar(df)

df['chop14'] = chop(df)
df['chop36'] = chop(df, lookback=36)

df['vo'] = vo(df['Volume'], short_lookback=18, long_lookback=36)

df['len_avg'], df['len_std'] = bar_lengths(abs(df['Mid_High'] - df['Mid_Low']))
df['sar_len_avg'], df['sar_len_std'] = sar_lengths(df['Mid_Open'], df['sar'])

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df_long['rsi'] = rsi(df_long)
df_long['slowk_rsi'], df_long['slowd_rsi'] = stoch_rsi(df_long['rsi'])

df_long['sar'] = psar(df_long)

df_long['chop14'] = chop(df_long)
df_long['chop36'] = chop(df_long, lookback=36)

df_long['vo'] = vo(df_long['Volume'], short_lookback=18, long_lookback=36)

df_long.dropna(inplace=True)
df_long.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-01-04 03:15:00   1.36925   1.36938  1.36903    1.36938   1.36944   
1     2021-01-04 03:20:00   1.36942   1.36944  1.36928    1.36938   1.36957   
2     2021-01-04 03:25:00   1.36937   1.36942  1.36935    1.36938   1.36954   
3     2021-01-04 03:30:00   1.36936   1.36945  1.36921    1.36936   1.36954   
4     2021-01-04 03:35:00   1.36934   1.36939  1.36924    1.36935   1.36953   
...                   ...       ...       ...      ...        ...       ...   
80471 2022-02-01 06:35:00   1.34495   1.34495  1.34471    1.34483   1.34518   
80472 2022-02-01 06:40:00   1.34487   1.34492  1.34460    1.34466   1.34507   
80473 2022-02-01 06:45:00   1.34469   1.34476  1.34446    1.34456   1.34490   
80474 2022-02-01 06:50:00   1.34458   1.34467  1.34450    1.34459   1.34475   
80475 2022-02-01 06:55:00   1.34456   1.34461  1.34422    1.34424   1.34477   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  ... 

In [8]:
value_per_pip = 1.0
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'JPY' else pips_to_risk * 100

    if second == 'USD':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'JPY' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def run_simulation(risk_reward_ratio, spread_cutoff, chop_cutoff, chop_num, use_rsi, strict_rsi, use_long_stoch, use_long_vo, use_half_sar, vo_cutoff, use_candles, use_spread_cushion, use_sar_distances, std_mult):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    trade = None

    for i in range(38, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        curr_long = df_long.loc[df_long['Date'] <= curr_date]
        if len(curr_long) < 2:
            continue
        ema200_2, ema100_2, ema50_2, ema25_2, atr2, sar2, rsi2, rsi_sma2, mid_open2, mid_close2, mid_low2, mid_high2, vo2 = df.loc[df.index[i - 2], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'rsi_sma', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High', 'vo']]
        ema200_1, ema100_1, ema50_1, ema25_1, atr1, sar1, rsi1, rsi_sma1, mid_open1, mid_close1, mid_low1, mid_high1, vo1 = df.loc[df.index[i - 1], ['ema200', 'ema100', 'ema50', 'ema25', 'atr', 'sar', 'rsi', 'rsi_sma', 'Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High', 'vo']]
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_mid_open, curr_mid_low = \
            df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                'Ask_Low', 'Ask_Close', 'Mid_Open', 'Mid_Low']]
        # chop2 = df.loc[df.index[i - 2], f'chop{chop_num}']
        # chop1 = df.loc[df.index[i - 1], f'chop{chop_num}']

        chop_long, vo_long, slowk_long, slowd_long = curr_long.loc[curr_long.index[-2], [f'chop{chop_num}', 'vo', 'slowk_rsi', 'slowd_rsi']]

        emas_buy_signal = ema200_1 < ema100_1
        emas_sell_signal = ema200_1 > ema100_1

        macd_buy_signal = macd2 < macdsignal2 and macd1 > macdsignal1 and max(macd_vals) == 0
        macd_sell_signal = macd2 > macdsignal2 and macd1 < macdsignal1 and min(macd_vals) == 0

        sar_buy_signal = sar1 < min([mid_low1, mid_high1])
        sar_sell_signal = sar1 > max([mid_low1, mid_high1])

        rsi_buy_signal = rsi1 > rsi_sma1 if use_rsi else True
        rsi_sell_signal = rsi1 < rsi_sma1 if use_rsi else True

        if strict_rsi and use_rsi:
            rsi_buy_signal = rsi_buy_signal and rsi1 > 50
            rsi_sell_signal = rsi_sell_signal and rsi1 < 50

        chop_signal = chop_long < chop_cutoff
        # chop_signal = True

        stoch_buy_signal = max([slowk_long, slowd_long, 20]) == 20 if use_long_stoch else True
        stoch_sell_signal = min([slowk_long, slowd_long, 80]) == 80 if use_long_stoch else True

        bullish_candle = mid_close1 > mid_open1 if use_candles else True
        bearish_candle = mid_close1 < mid_open1 if use_candles else True


        if not use_sar_distances:
            mean, std = df.loc[df.index[i - 3], ['len_avg', 'len_std']]

            bars_small_enough = abs(mid_high1 - mid_low1) <= mean + (std_mult * std) and abs(mid_high2 - mid_low2) <= mean + (std_mult * std)

        else:
            mean, std = df.loc[df.index[i - 1], ['sar_len_avg', 'sar_len_std']]

            bars_small_enough = abs(curr_mid_open - sar1) <= mean + (std / 1)

        # bars_small_enough = True

        buy_limit = curr_ask_low <= curr_ask_open
        sell_limit = curr_bid_open <= curr_bid_high

        vo_signal = ((vo2 + vo1) / 2 > vo_cutoff and vo_long > vo_cutoff) if use_long_vo else ((vo2 + vo1) / 2 > vo_cutoff)

        if trade is None:
            if macd_buy_signal and emas_buy_signal and sar_buy_signal and rsi_buy_signal and chop_signal and stoch_buy_signal and bullish_candle and bars_small_enough and buy_limit and vo_signal:
                open_price = float(curr_ask_open)
                pullback = mid_low1 - abs(mid_low1 - sar1) / 2 if use_half_sar else sar1
                pullback -= spread if use_spread_cushion else 0

                stop_loss = round(pullback, 5)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                        # stop_gain = round(open_price + 0.0005, 5)

                        n_units = get_n_units('buy', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, 'AUD_USD')

                        trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                        'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_buys += 1

                        pips_risked.append(curr_pips_to_risk)

            elif macd_sell_signal and emas_sell_signal and sar_sell_signal and rsi_sell_signal and chop_signal and stoch_sell_signal and bearish_candle and bars_small_enough and sell_limit and vo_signal:
                open_price = float(curr_bid_open)
                pullback = abs(sar1 - mid_high1) / 2 + mid_high1 if use_half_sar else sar1
                pullback += spread if use_spread_cushion else 0

                stop_loss = round(pullback, 5)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                        # stop_gain = round(open_price - 0.0005, 5)

                        n_units = get_n_units('sell', stop_loss, curr_ask_open, curr_bid_open, curr_mid_open, 'AUD_USD')

                        trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                        n_sells += 1

                        pips_risked.append(curr_pips_to_risk)


        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None    

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [12]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
# risk_reward_ratio_vals = [1.5, 2]
risk_reward_ratio_vals = [1.5]
# spread_cutoffs = [0.15, 0.20]
spread_cutoffs = [0.20]
# chop_cutoff_vals = [0.50, 0.559, 0.618]
chop_cutoff_vals = [0.50]
# chop_nums = [14, 36]
chop_nums = [36]
use_rsi_vals = [True, False]
use_strict_rsi_vals = [True, False]
# use_rsi_vals = [True]
use_long_stoch_vals = [True, False]
use_long_vo_vals = [True, False]
# use_half_sar_vals = [True, False]
use_half_sar_vals = [False]
# vo_cutoff_vals = [0.0, 0.10]
vo_cutoff_vals = [0.0]
use_candle_vals = [True, False]
# use_candle_vals = [True]
use_spread_cushion_vals = [True, False]
# use_spread_cushion_vals = [True]
use_sar_distances_vals = [True, False]
# use_sar_distances_vals = [False]
std_mult_vals = [1.0, 2.0]
# std_mult_vals = [2.0]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(chop_cutoff_vals) * len(chop_nums) * len(use_rsi_vals) * len(use_strict_rsi_vals) * len(use_long_stoch_vals) * len(use_long_vo_vals) * len(use_half_sar_vals) * len(vo_cutoff_vals) * len(use_candle_vals) * len(use_spread_cushion_vals) * len(use_sar_distances_vals) * len(std_mult_vals)
all_combos = []


# use_half_sar, vo_cutoff, use_candles

for risk_reward_ratio in risk_reward_ratio_vals:
    for spread_val in spread_cutoffs:
        for chop_cutoff in chop_cutoff_vals:
            for chop_num in chop_nums:
                for use_rsi in use_rsi_vals:
                    for strict_rsi in use_strict_rsi_vals:
                        for use_long_stoch in use_long_stoch_vals:
                            for use_long_vo in use_long_vo_vals:
                                for use_half_sar in use_half_sar_vals:
                                    for vo_cutoff in vo_cutoff_vals:
                                        for use_candle in use_candle_vals:
                                            for use_spread_cushion in use_spread_cushion_vals:
                                                for use_sar_distances in use_sar_distances_vals:
                                                    for std_mult in std_mult_vals:
                                                        all_combos.append((risk_reward_ratio, spread_val, chop_cutoff, chop_num, use_rsi, strict_rsi, use_long_stoch, use_long_vo, use_half_sar, vo_cutoff, use_candle, use_spread_cushion, use_sar_distances, std_mult))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_chop_cutoff_val = None
best_chop_num = None
best_use_rsi_val = None
best_strict_rsi_val = None
best_use_long_stoch_val = None
best_use_long_vo_val = None
best_use_half_sar_val = None
best_vo_cutoff = None
best_use_candle_val = None
best_use_spread_cushion_val = None
best_use_sar_distances_val = None
best_std_mult = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_cutoff, chop_cutoff, chop_num, use_rsi, strict_rsi, use_long_stoch, use_long_vo, use_half_sar, vo_cutoff, use_candles, use_spread_cushion, use_sar_distances, std_mult in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_cutoff, chop_cutoff, chop_num, use_rsi, strict_rsi, use_long_stoch, use_long_vo, use_half_sar, vo_cutoff, use_candles, use_spread_cushion, use_sar_distances, std_mult)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_buys))
    print('Num sells: ' + str(n_sells))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    # print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread_cutoff': spread_cutoff, 'chop_cutoff': chop_cutoff, 'chop_num': chop_num, 'use_rsi': use_rsi, 'strict_rsi': strict_rsi, 'use_long_stoch': use_long_stoch, 'use_long_vo': use_long_vo, 'use_half_sar': use_half_sar, 'vo_cutoff': vo_cutoff, 'use_candles': use_candles, 'use_spread_cushion': use_spread_cushion, 'use_sar_distances': use_sar_distances, 'std_mult': std_mult})


    if reward > best_reward:
        best_reward = reward
        best_risk_reward = risk_reward_ratio
        best_spread_cutoff = spread_val
        best_chop_cutoff_val = chop_cutoff
        best_chop_num = chop_num
        best_use_rsi_val = use_rsi
        best_strict_rsi_val = strict_rsi
        best_use_long_stoch_val = use_long_stoch
        best_use_long_vo_val = use_long_vo
        best_use_half_sar_val = use_half_sar
        best_vo_cutoff = vo_cutoff
        best_use_candle_val = use_candles
        best_use_spread_cushion_val = use_spread_cushion
        best_use_sar_distances_val = use_sar_distances
        best_std_mult = std_mult

    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 256

-1686.507469999965
Num buys: 46
Num sells: 50
Num trades: 96
Num wins: 25
Num losses: 71
Win streak: 3
Loss streak: 8
Remaining runs: 255
Best reward so far: -1686.507469999965

-832.8086499999772
Num buys: 20
Num sells: 29
Num trades: 49
Num wins: 13
Num losses: 36
Win streak: 2
Loss streak: 9
Remaining runs: 254
Best reward so far: -832.8086499999772

-99.99659999999672
Num buys: 4
Num sells: 3
Num trades: 7
Num wins: 2
Num losses: 5
Win streak: 1
Loss streak: 3
Remaining runs: 253
Best reward so far: -99.99659999999672

-849.8256200000116
Num buys: 24
Num sells: 18
Num trades: 42
Num wins: 10
Num losses: 32
Win streak: 3
Loss streak: 9
Remaining runs: 252
Best reward so far: -99.99659999999672

-649.9052700000082
Num buys: 11
Num sells: 7
Num trades: 18
Num wins: 2
Num losses: 16
Win streak: 1
Loss streak: 10
Remaining runs: 251
Best reward so far: -99.99659999999672

-829.0197499999978
Num buys: 23
Num sells: 21
Num trades: 44
Num wins: 11
Num losses: 33
Win streak: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best chop cutoff val: ' + str(best_chop_cutoff_val))
print('Best chop val: ' + str(best_chop_num))
print('Best use rsi val: ' + str(best_use_rsi_val))
print('Best strict rsi val: ' + str(best_strict_rsi_val))
print('Best use long stoch val: ' + str(best_use_long_stoch_val))
print('Best use long vo val: ' + str(best_use_long_vo_val))
print('Best use half sar val: ' + str(best_use_half_sar_val))
print('Best vo cutoff: ' + str(best_vo_cutoff))
print('Best use candle val: ' + str(best_use_candle_val))
print('Best use spread cushion val: ' + str(best_use_spread_cushion_val))
print('Best use sar distances val: ' + str(best_use_sar_distances_val))
print('Best std mult: ' + str(best_std_mult))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 461.937110000008
Best risk/reward ratio: 1.5
Best spread: 0.2
Best chop cutoff val: 0.5
Best chop val: 36
Best use rsi val: True
Best use long ema val: False
Best use any macd val: False
Best use half sar val: False
Best vo cutoff: 0.0
Best use candle val: True
Best use spread cushion val: False
Best use sar distances val: False
Best std mult: 2.0
-----------------------
Top results:
{'reward': 131, 'ratio': 1.5, 'spread': 0.2, 'chop_cutoff': 0.5, 'chop_num': 14, 'use_rsi': True, 'use_long_ema': False, 'use_any_macd': False, 'use_half_sar': False, 'vo_cutoff': 0.0, 'use_candle': True, 'use_spread_cushion': False, 'use_sar_distances': False, 'std_mult': 2.0}
{'reward': 398, 'ratio': 1.5, 'spread': 0.2, 'chop_cutoff': 0.5, 'chop_num': 14, 'use_rsi': True, 'use_long_ema': False, 'use_any_macd': False, 'use_half_sar': False, 'vo_cutoff': 0.0, 'use_candle': False, 'use_spread_cushion': False, 'use_sar_distances': True, 'std_mult': 1.5}
{'